In [16]:
import os
import subprocess
import pandas as pd
import numpy as np

## Определяем кластеры

In [14]:
folders = ["data/" + folder + "/*" for folder in os.listdir("data")]
" ".join(folders)

'data/Halomonas/* data/Enterobacter/* data/Mannheimia/* data/Aeromonas/* data/Erwinia/* data/Lysobacter/* data/Klebsiella/* data/Dickeya/* data/Kosakonia/* data/Pseudomonas/*'

In [17]:
subprocess.run("proteinortho5 -project=gammaproteo " + " ".join(folders), shell=True, check=True)

CompletedProcess(args='proteinortho5 -project=gammaproteo data/Halomonas/* data/Enterobacter/* data/Mannheimia/* data/Aeromonas/* data/Erwinia/* data/Lysobacter/* data/Klebsiella/* data/Dickeya/* data/Kosakonia/* data/Pseudomonas/*', returncode=0)

In [18]:
! mv gammaproteo* proteinortho/